In [2]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
#ignore all Depricated Warning
simplefilter(action='ignore', category=DeprecationWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=RuntimeWarning)
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle


In [3]:
train_data = pd.read_csv("F:/Project/application_train.csv")

In [4]:
print('Training data shape: ', train_data.shape)

Training data shape:  (307511, 122)


In [22]:
dum_train = pd.get_dummies(train_data, drop_first=True)
dum1=dum_train.dropna(axis=1)
X = dum1.drop('TARGET',axis=1)
y = dum_train['TARGET']

In [23]:
print(y)

0         1
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        1
27        0
28        0
29        0
         ..
307481    1
307482    0
307483    0
307484    0
307485    0
307486    0
307487    0
307488    0
307489    1
307490    0
307491    0
307492    0
307493    0
307494    0
307495    0
307496    0
307497    0
307498    0
307499    0
307500    0
307501    0
307502    0
307503    0
307504    0
307505    0
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int64


In [24]:
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [25]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 2019)

In [26]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [27]:
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 50


In [28]:
clf = lgb.train(params, d_train, 100)
print("Completed")

Completed


In [29]:
y_pred=clf.predict(x_test)

In [30]:
for i in range(0,99):
    if y_pred[i]>=.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

In [34]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred.round())

In [32]:
y_pred

array([0.        , 0.        , 0.        , ..., 0.08259675, 0.09746013,
       0.0748256 ])

In [37]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred.round(),y_test)

In [38]:
accuracy

0.9186159949704078

In [39]:
ls 


 Volume in drive C has no label.
 Volume Serial Number is 0E99-3D4B

 Directory of C:\Users\dbda

07/17/2019  07:27 PM    <DIR>          .
07/17/2019  07:27 PM    <DIR>          ..
05/29/2019  06:29 PM    <DIR>          .anaconda
05/15/2019  06:10 PM               122 .bash_history
07/17/2019  11:17 AM    <DIR>          .conda
05/31/2019  09:44 AM                44 .condarc
03/25/2019  12:34 PM    <DIR>          .continuum
03/23/2019  01:07 PM            17,954 .dbshell
03/07/2019  06:56 PM    <DIR>          .eclipse
03/15/2019  04:42 PM    <DIR>          .idlerc
07/17/2019  02:44 PM    <DIR>          .ipynb_checkpoints
03/23/2019  12:33 PM    <DIR>          .ipython
03/26/2019  04:19 PM    <DIR>          .jupyter
03/14/2019  04:10 PM    <DIR>          .m2
06/28/2019  08:03 PM    <DIR>          .matplotlib
03/08/2019  07:30 PM                 0 .mongorc.js
04/03/2019  02:52 PM    <DIR>          .p2
03/07/2019  06:56 PM    <DIR>          .scalaide
07/17/2019  06:38 PM    <DIR>          